# Short-Circuit

Let's see how we can make a short-circuit calculation.

In [ ]:
import getpass

import numpy as np

from roseau.load_flow import *

We will start by creating a simple network composed of two LV lines.
As usual with short-circuit calculations, we won't add any loads.

<div class="alert alert-info">

**Note:**
While impedance and current loads could technically be added to the network, it is not possible to add a power load on the same bus as the one we want to compute the short-circuit on. This is because having `I = (S/U)*` with `U=0` is impossible.

</div>

In [ ]:
# Create three buses
source_bus = Bus(id="sb", phases="abcn")
bus1 = Bus(id="b1", phases="abcn")
bus2 = Bus(id="b2", phases="abcn")

# Define the reference of potentials
ground = Ground(id="gnd")
pref = PotentialRef(id="pref", element=ground)
ground.connect(bus=source_bus)

# Create a LV source at the first bus
un = 400 / np.sqrt(3)
source_voltages = [un, un * np.exp(-2j * np.pi / 3), un * np.exp(2j * np.pi / 3)]
vs = VoltageSource(id="vs", bus=source_bus, phases="abcn", voltages=source_voltages)

# Add LV lines
lp1 = LineParameters.from_name_lv("S_AL_240")
line1 = Line(id="line1", bus1=source_bus, bus2=bus1, parameters=lp1, length=1.0, ground=ground)
lp2 = LineParameters.from_name_lv("S_AL_150")
line2 = Line(id="line2", bus1=bus1, bus2=bus2, parameters=lp2, length=2.0, ground=ground)

# Create network
en = ElectricalNetwork.from_element(source_bus)

## Phase-to-phase

We can now add a short-circuit. Let's first create a phase-to-phase short-circuit:

In [ ]:
bus2.add_short_circuit("a", "b")

Let's run the load flow, and get the current results.

In [ ]:
# Authentication
auth = (input("Please enter your username:"), getpass.getpass("Please enter your password:"))

In [ ]:
en.solve_load_flow(auth=auth)
en.res_branches

Looking at the line results of the second bus of the line "line2", which is "bus2" where we
added the short-circuit, one can notice that:

* the potentials of phases "a" and "b" are equal;
* the currents and powers in phases "a" and "b" are equal with opposite signs, i.e. the sum of the currents is zero;
* the currents and powers in these two phases are very high;

which is expected from a short-circuit.

## Multi-phase

It is possible to create short-circuits between several phases, not only two. Let's first
remove the existing short-circuit then create a new one between phases "a", "b", and "c".

In [ ]:
bus2.clear_short_circuits()
bus2.add_short_circuit("a", "b", "c")
en.solve_load_flow(auth=auth)
en.res_branches

Now the potentials of the three phases are equal and the currents and powers add up to
zero at the bus where the short-circuit is applied.

## Phase-to-ground

Phase-to-ground short-circuits are also possible. Let's remove the existing short-circuit
and create a new one between phase "a" and ground.

In [ ]:
bus2.clear_short_circuits()
bus2.add_short_circuit("a", ground=ground)  # ground MUST be passed as a keyword argument
en.solve_load_flow(auth=auth)
en.res_branches

In [ ]:
en.res_grounds

Here the potential at phase "a" of bus "bus2" is zero, equal to the ground potential.
The sum of the currents in the other phases is also zero indicating that the current
of phase "a" went through the ground.

## Additional notes

The library will prevent the user from making mistakes, for example when trying to add a power load with the short-circuit, or when forgetting parameters.

In [ ]:
try:
    load = PowerLoad("load", bus=bus2, powers=[10, 10, 10])
except RoseauLoadFlowException as e:
    print(e)

In [ ]:
try:
    bus2.add_short_circuit("a")
except RoseauLoadFlowException as e:
    print(e)